### Librerias

In [45]:
import time
from datetime import datetime
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium import webdriver
import pprint
import pandas as pd
import names as nm
import numpy as np
import traceback

###  Código

In [51]:
def click_boton(xpath, nombre_auxilliar="marca"):
    "dado un full xpath hace click"
    #chequea que el elemento exista
    elemento  = WebDriverWait(driver, 15).until( \
        EC.presence_of_element_located((By.XPATH, f"{xpath}")))
    time.sleep(0.8)
    elemento.click()

    
def escribir_y_enter(xpath, texto="", sleep_time=0):
    "dado un full xpath hace click, envía un texto y apreta enter"
    elemento = WebDriverWait(driver, 10).until( \
        EC.presence_of_element_located((By.XPATH, f"{xpath}")))
    time.sleep(sleep_time) #duerme un tiempo
    elemento.click()
    elemento.send_keys(texto, Keys.ENTER)
    

def soup_of_page(driver):
    time.sleep(2.5)
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    return soup

def generar_persona():
    '''return 
    sexo, nombre, apellido, email, numero'''
    #nombre y apeelido
    sexo =np.random.binomial(n=1, p=0.5)
    sexo = np.where(sexo==0, "male", "female" )
    random_number= np.random.randint(0,9999, 1)[0]
    nombre = nm.get_first_name(gender=sexo)
    apellido =  nm.get_last_name()
    fin_mail= "@gmail.com"
    email = nombre+apellido+str(random_number)+fin_mail
    email = email.lower()
    numero = np.random.randint(0,9,size=8)
    numero_str = ''.join([str(elem) for elem in numero])

    return sexo, nombre, apellido, email, numero_str


def chequera_indice_provincia(html_tags, key):
    '''busca la provincia key en el array de html tags
    y devuelve el indice'''
    prov = key.strip().capitalize()
    for idx, provi in  enumerate(html_tags):
        res  = provi.get_text()
        res = str(res)
        res = res.strip().capitalize()
        if res == prov:
            return idx+1
    return "no encontrado"

In [54]:
options = Options()
options.add_argument("--window-size=1980,1080")
options.add_argument("start-maximized")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
#options.add_argument("--headless")
driver = webdriver.Chrome(options=options)

nuevo = True

def get_url_perfil(marca, año, modelo, fecha, version ,  provincia, localidad, nuevo, id_perfil):
    tic = time.perf_counter()
    driver.get('https://www.123seguro.com.ar/seguros/auto/cotizar/marca')
    time.sleep(6)
    # Selección marca
    mas_marcas_path = "/html/body/div[1]/section/main/section[2]/section/div/section/section/div[2]/button"
    click_boton(mas_marcas_path)

    # Busqueda de Marca Particular
    soup = soup_of_page(driver)
    divs_marcas = soup.find_all("button", {"class":"ui-button-selector"})
    marcas = [ x.get_text().strip().capitalize()  for x in divs_marcas  ]
    marca = marca.strip().capitalize()
    idx_marca = [key+1 for key, val in enumerate(marcas)if val in [marca]][0]

    marca_path= f"/html/body/div[1]/section/main/section[2]/section/div/section/section/div[2]/div/button[{idx_marca}]"
    click_boton(marca_path)


    # Provincia
    soup = soup_of_page(driver)
    divs_provicinas = soup.find_all("button", {"class":"ui-button-selector"})
    idx_provincia = chequera_indice_provincia(divs_provicinas, "Capital Federal")


    # Año 
    año_path = "/html/body/div[1]/section/main/section[2]/section/div/section/div[1]/div[2]/div/div/input"
    escribir_y_enter(año_path, texto = año)


    # Modelo 
    modelo_path = '/html/body/div[1]/section/main/section[2]/section/div/section/div/div[1]/div[2]/div/div[2]/div/input'
    modelo = modelo.strip().capitalize()
    escribir_y_enter(modelo_path, texto= modelo, sleep_time=1.5)
    try: 
        click_boton("/html/body/div[1]/section/main/section[2]/section/div/section/div/div[2]/div/div/div/button[1]") 
    except:
        print("el modelo se encontro!")
    
    # Especifico
    print(version)
    especifico_path = '/html/body/div[1]/section/main/section[2]/section/div/section/div[1]/div[2]/div/div/input'
    escribir_y_enter(xpath=especifico_path, texto=version, sleep_time=3)
    #seleccionar primera opción
    click_boton("/html/body/div[1]/section/main/section[2]/section/div/section/div[2]/div[1]/div/button[1]")
    
    if año=="2021":
        if nuevo:
            click_boton("/html/body/div[1]/section/main/section[2]/section/div/section/div[2]/button[1]")

        else:
            click_boton("/html/body/div[1]/section/main/section[2]/section/div/section/div[2]/button[2]")

    #continuar
    click_boton('/html/body/div[1]/section/main/section[2]/section/div/section/div[2]/button')

    
    #persona 
    sexo, nombre, apellido, email, numero = generar_persona()

    #Nombre y Apellido
    nombre_path = '/html/body/div[1]/section/main/section[2]/section/div/section/div/div[2]/div/form/div[1]/div/input'
    escribir_y_enter(nombre_path,nombre, sleep_time=1 )

    apellido_path ='/html/body/div[1]/section/main/section[2]/section/div/section/div/div[2]/div/form/div[2]/div/input'
    escribir_y_enter(apellido_path, apellido, sleep_time=1)


    #Fecha 
    fecha_path = '/html/body/div[1]/section/main/section[2]/section/div/section/div[2]/div/form/div/div/div/input'
    escribir_y_enter(fecha_path, texto=fecha, sleep_time=1.4 )

    #Provincias
    click_boton('/html/body/div[1]/section/main/section[2]/section/div/section/div[1]/div[2]/div/div')
    time.sleep(1)

    #Busqueda de Opciones de provincia
    soup = soup_of_page(driver)
    divs_provicinas = soup.find_all("div", {"class":"option"})
    idx_provincia = chequera_indice_provincia(divs_provicinas, provincia)
    click_boton(f"/html/body/div[1]/section/main/section[2]/section/div/section/div[1]/div[2]/div[1]/div[2]/div[{idx_provincia}]") #Volvemos con Selenium

    #LOCALIDAD
    locacion_path = '//*[@id="location-selection"]/div[1]/div[2]/div[2]/div[1]/div/input'
    escribir_y_enter( '//*[@id="location-selection"]/div[1]/div[2]/div[2]/div[1]/div/input', texto=localidad, sleep_time=2)

    #Mail
    mail_path = '/html/body/div[1]/section/main/section[2]/section/div/section/div[2]/div/form/div/div/input'
    escribir_y_enter(mail_path,email )

    #Telefono
    cod_area_path ='/html/body/div[1]/section/main/section[2]/section/div/section/div[2]/div/form/div[2]/div[2]/div/input'
    escribir_y_enter(cod_area_path, texto="11")

    telefono_path = '/html/body/div[1]/section/main/section[2]/section/div/section/div[2]/div/form/div[2]/div[3]/div/input'
    escribir_y_enter(telefono_path, numero)

    time.sleep(12)

    url = driver.current_url
    
    toc =  time.perf_counter()
    runtime = toc-tic
    print(f"*[{id_perfil}] - Cargar el url llevo {runtime} segundos!")
    
    return url

In [55]:
def testear_un_caso(_id):
    """Comprueba como funciona el scrap para un id de la tabla del excel"""
    data = pd.read_excel("./perfiles.xlsx", sheet_name="perfiles")
    df = data[data["id"]==_id]
    data_dict= dict(data.loc[_id, :])
    valores = data_dict.values()
    params = [str(v) for v in valores]
    id_perfil, marca, año, modelo, version, \
    fecha, provincia, localidad = params

    url = get_url_perfil(marca, año, modelo, fecha, version, provincia, localidad, nuevo, id_perfil)

In [56]:
def get_urls_all_perfiles(direccion_excel):
    data = pd.read_excel(direccion_excel, sheet_name="perfiles")
    cantidad_perfiles =  data.shape[0]
    urls = {}
    
    print(f"* El tiempo estimado de Ejecución Total son {46*cantidad_perfiles} segundos 😘")
    for i in range(cantidad_perfiles):
        try:
            data_dict= dict(data.loc[i, :])
            valores = data_dict.values()
            params = [str(v) for v in valores]
            id_perfil, marca, año, modelo, version, \
            fecha, provincia, localidad = params
            url = get_url_perfil(marca, año, modelo, fecha, version, provincia, localidad, nuevo, id_perfil)
            urls[f"{id_perfil}"]= url
        except:
            traceback.print_exc()
            print(f"* El perfil {id_perfil} Fallo, continuando con el siguiente")
        
        print("\nLa lista de urls está así:")
        pprint.pprint(urls)
    return urls

In [57]:
urls = get_urls_all_perfiles(direccion_excel="./perfiles.xlsx")

* El tiempo estimado de Ejecución Total son 230 segundos 😘
el modelo se encontro!
CAMARO SS L/17 de CAMARO


Traceback (most recent call last):
  File "<ipython-input-56-8a6bce7897f4>", line 14, in get_urls_all_perfiles
    url = get_url_perfil(marca, año, modelo, fecha, version, provincia, localidad, nuevo, id_perfil)
  File "<ipython-input-54-d680fb793f7e>", line 55, in get_url_perfil
    click_boton("/html/body/div[1]/section/main/section[2]/section/div/section/div[2]/div[1]/div/button[1]")
  File "<ipython-input-51-91527aa96df1>", line 4, in click_boton
    elemento  = WebDriverWait(driver, 15).until( \
  File "C:\Users\Matias\anaconda3\lib\site-packages\selenium\webdriver\support\wait.py", line 80, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 



* El perfil 1 Fallo, continuando con el siguiente

La lista de urls está así:
{}
el modelo se encontro!
L/16 1.6 4 P S


Traceback (most recent call last):
  File "<ipython-input-56-8a6bce7897f4>", line 14, in get_urls_all_perfiles
    url = get_url_perfil(marca, año, modelo, fecha, version, provincia, localidad, nuevo, id_perfil)
  File "<ipython-input-54-d680fb793f7e>", line 55, in get_url_perfil
    click_boton("/html/body/div[1]/section/main/section[2]/section/div/section/div[2]/div[1]/div/button[1]")
  File "<ipython-input-51-91527aa96df1>", line 4, in click_boton
    elemento  = WebDriverWait(driver, 15).until( \
  File "C:\Users\Matias\anaconda3\lib\site-packages\selenium\webdriver\support\wait.py", line 80, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 



* El perfil 2 Fallo, continuando con el siguiente

La lista de urls está así:
{}
2.0 XEI L/20 CVT


Traceback (most recent call last):
  File "<ipython-input-56-8a6bce7897f4>", line 14, in get_urls_all_perfiles
    url = get_url_perfil(marca, año, modelo, fecha, version, provincia, localidad, nuevo, id_perfil)
  File "<ipython-input-54-d680fb793f7e>", line 55, in get_url_perfil
    click_boton("/html/body/div[1]/section/main/section[2]/section/div/section/div[2]/div[1]/div/button[1]")
  File "<ipython-input-51-91527aa96df1>", line 4, in click_boton
    elemento  = WebDriverWait(driver, 15).until( \
  File "C:\Users\Matias\anaconda3\lib\site-packages\selenium\webdriver\support\wait.py", line 80, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 



* El perfil 3 Fallo, continuando con el siguiente

La lista de urls está así:
{}
el modelo se encontro!
L/16 2.4 DC 4x4 TDI DX


Traceback (most recent call last):
  File "<ipython-input-56-8a6bce7897f4>", line 14, in get_urls_all_perfiles
    url = get_url_perfil(marca, año, modelo, fecha, version, provincia, localidad, nuevo, id_perfil)
  File "<ipython-input-54-d680fb793f7e>", line 55, in get_url_perfil
    click_boton("/html/body/div[1]/section/main/section[2]/section/div/section/div[2]/div[1]/div/button[1]")
  File "<ipython-input-51-91527aa96df1>", line 4, in click_boton
    elemento  = WebDriverWait(driver, 15).until( \
  File "C:\Users\Matias\anaconda3\lib\site-packages\selenium\webdriver\support\wait.py", line 80, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 



* El perfil 4 Fallo, continuando con el siguiente

La lista de urls está así:
{}
el modelo se encontro!
1.4 LT L/17
* El perfil 5 Fallo, continuando con el siguiente

La lista de urls está así:
{}


Traceback (most recent call last):
  File "<ipython-input-56-8a6bce7897f4>", line 14, in get_urls_all_perfiles
    url = get_url_perfil(marca, año, modelo, fecha, version, provincia, localidad, nuevo, id_perfil)
  File "<ipython-input-54-d680fb793f7e>", line 55, in get_url_perfil
    click_boton("/html/body/div[1]/section/main/section[2]/section/div/section/div[2]/div[1]/div/button[1]")
  File "<ipython-input-51-91527aa96df1>", line 4, in click_boton
    elemento  = WebDriverWait(driver, 15).until( \
  File "C:\Users\Matias\anaconda3\lib\site-packages\selenium\webdriver\support\wait.py", line 80, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 



In [37]:
urls={'1': 'https://cotizador.123seguro.com.ar/cotizar-auto/lead/1026957/#cotizaciones',
 '2': 'https://cotizador.123seguro.com.ar/cotizar-auto/lead/1026959/#',
 '3': 'https://cotizador.123seguro.com.ar/cotizar-auto/lead/1026960/#',
 '4': 'https://cotizador.123seguro.com.ar/cotizar-auto/lead/1026961/#cotizaciones/info-personal#lead',
 '5': 'https://cotizador.123seguro.com.ar/cotizar-auto/lead/1026963/#'}

In [42]:
def guardar_urls(urls):
    df = pd.DataFrame(urls, index=["Seguros"])
    df = df.T
    df = df.reset_index()
    df = df.rename(columns={"index":"Id"})
    df.to_excel('./urls.xlsx', sheet_name="urls", index=False)

In [43]:
guardar_urls(urls, )